In [1]:
from dispersion import *
from fitter import *
from likelihood import *
from profiles import *
from utils import *

In [2]:
dm = ZhaoProfile()
st = genPlummerProfile()
kr = AnisotropyKernel(**{'model':'constBeta'})

In [3]:
kr.__dict__

{'beta': 0.0, 'model': 'constBeta', 'params': ['beta']}

In [4]:
Sigma = SphericalJeansDispersion(dm, st, kr)

In [5]:
Sigma.params

{'dm_a': 1,
 'dm_b': 3,
 'dm_c': 1,
 'dm_r0': 1,
 'dm_rho0': 1,
 'ker_beta': 0.0,
 'st_c': 0,
 'st_rh': 1,
 'st_rhoh': 1}

In [21]:
LL = LogLikelihood(np.arange(1,4),Sigma)

In [22]:
LL.set_free('dm_c')

In [23]:
LL.sigma.kernel.model = 'OM'

In [24]:
LL.sigma.params

{'dm_a': 1,
 'dm_b': 3,
 'dm_c': 1,
 'dm_r0': 1,
 'dm_rho0': 1,
 'ker_beta': 0.0,
 'st_c': 0,
 'st_rh': 1,
 'st_rhoh': 1}

In [26]:
LL.set_free('ra')

ValueError: ra not a parameter of logLike function.
The parameters are ['st_rh', 'dm_r0', 'st_c', 'dm_a', 'dm_b', 'dm_c', 'dm_rho0', 'st_rhoh', 'ker_beta']

In [8]:
M = MinuitFitter(LL)

In [11]:
M.loglike.sigma.setparams('st_c',0)